# Seq2SeqRepresentation - Training

In [1]:
import os, datetime
import numpy as np
import joblib
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer,text_to_word_sequence

## Collect HTML files

Load the HTML files

In [2]:
files = []

for f in os.listdir(os.path.join(os.getcwd(),'html')):
    fd = open(os.path.join(os.getcwd(),'html',f),'r')
    content = fd.read()
    files.append(content)
    fd.close()

Add a mock file as a concatentation of a previous one

In [3]:
files.append(files[0]*10)

## Preprocess the files

Define the padding and trimming parameters

In [4]:
padding='post'

maxlen=200
truncating='post'

Load the tokenizer

In [5]:
SQLi_tokenizer = joblib.load('ignore_tokenizer_20210318153153509112')

Convert the pages into token sequences:

In [6]:
tokens_list = SQLi_tokenizer.texts_to_sequences(files)

Pad and trim the sequences to fit them into an array:

In [7]:
tokens_array = keras.preprocessing.sequence.pad_sequences(tokens_list, padding=padding, maxlen=maxlen, truncating=truncating)

## Generating a target vector for teacher forcing

We set up a shifted (by -1) version of the input to allow the model to learn a step ahead.

In [8]:
target_array = np.zeros_like(tokens_array)
target_array[:,1:] = tokens_array[:,0:-1]

## Converting into one-hot encoding

Translating integer encoding into one-hot.

In [9]:
tokens_1hot_array = keras.utils.to_categorical(tokens_array)
target_1hot_array = keras.utils.to_categorical(target_array)

In [10]:
tokens_1hot_array.shape

(5, 200, 55)

## Defining Seq2Seq

Instantiating a VAE module built following [https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html](https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html).

In [11]:
latent_dim = 3
num_tokens = tokens_1hot_array.shape[2]

encoder_inputs = Input(shape=(None, num_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outpus, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,num_tokens))
decoder = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs,_,_ = decoder(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_tokens,activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = keras.Model([encoder_inputs,decoder_inputs],decoder_outputs)
model.summary()    

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 55)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 55)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 3), (None, 3 708         input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 3), (N 708         input_2[0][0]                    
                                                                 lstm[0][1]            

## Training the Seq2Seq

Instantiating and training the model

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([tokens_1hot_array,tokens_1hot_array],target_1hot_array,batch_size=5,epochs=10000)

Epoch 1/10000
1/1 [==============================] - 0s 0s/step - loss: 4.0409
Epoch 2/10000
1/1 [==============================] - 0s 0s/step - loss: 4.0332
Epoch 3/10000
1/1 [==============================] - 0s 1ms/step - loss: 4.0279
Epoch 4/10000
1/1 [==============================] - 0s 0s/step - loss: 4.0237
Epoch 5/10000
1/1 [==============================] - 0s 1ms/step - loss: 4.0200
Epoch 6/10000
1/1 [==============================] - 0s 0s/step - loss: 4.0167
Epoch 7/10000
1/1 [==============================] - 0s 1ms/step - loss: 4.0137
Epoch 8/10000
1/1 [==============================] - 0s 1ms/step - loss: 4.0108
Epoch 9/10000
1/1 [==============================] - 0s 0s/step - loss: 4.0081
Epoch 10/10000
1/1 [==============================] - 0s 1ms/step - loss: 4.0055
Epoch 11/10000
1/1 [==============================] - 0s 0s/step - loss: 4.0030
Epoch 12/10000
1/1 [==============================] - 0s 1ms/step - loss: 4.0006
Epoch 13/10000
1/1 [=======================

1/1 [==============================] - 0s 1ms/step - loss: 3.7889
Epoch 103/10000
1/1 [==============================] - 0s 1ms/step - loss: 3.7860
Epoch 104/10000
1/1 [==============================] - 0s 1ms/step - loss: 3.7831
Epoch 105/10000
1/1 [==============================] - 0s 1ms/step - loss: 3.7801
Epoch 106/10000
1/1 [==============================] - 0s 1ms/step - loss: 3.7772
Epoch 107/10000
1/1 [==============================] - 0s 1ms/step - loss: 3.7742
Epoch 108/10000
1/1 [==============================] - 0s 1ms/step - loss: 3.7712
Epoch 109/10000
1/1 [==============================] - 0s 1ms/step - loss: 3.7682
Epoch 110/10000
1/1 [==============================] - 0s 0s/step - loss: 3.7652
Epoch 111/10000
1/1 [==============================] - 0s 1ms/step - loss: 3.7621
Epoch 112/10000
1/1 [==============================] - 0s 1ms/step - loss: 3.7591
Epoch 113/10000
1/1 [==============================] - 0s 0s/step - loss: 3.7560
Epoch 114/10000
1/1 [=============

1/1 [==============================] - 0s 1ms/step - loss: 3.4157
Epoch 203/10000
1/1 [==============================] - 0s 0s/step - loss: 3.4115
Epoch 204/10000
1/1 [==============================] - 0s 1ms/step - loss: 3.4072
Epoch 205/10000
1/1 [==============================] - 0s 0s/step - loss: 3.4030
Epoch 206/10000
1/1 [==============================] - 0s 0s/step - loss: 3.3988
Epoch 207/10000
1/1 [==============================] - 0s 1ms/step - loss: 3.3945
Epoch 208/10000
1/1 [==============================] - 0s 1ms/step - loss: 3.3903
Epoch 209/10000
1/1 [==============================] - 0s 994us/step - loss: 3.3861
Epoch 210/10000
1/1 [==============================] - 0s 559us/step - loss: 3.3819
Epoch 211/10000
1/1 [==============================] - 0s 0s/step - loss: 3.3776
Epoch 212/10000
1/1 [==============================] - 0s 1ms/step - loss: 3.3734
Epoch 213/10000
1/1 [==============================] - 0s 0s/step - loss: 3.3692
Epoch 214/10000
1/1 [============

1/1 [==============================] - 0s 0s/step - loss: 2.9990
Epoch 303/10000
1/1 [==============================] - 0s 0s/step - loss: 2.9949
Epoch 304/10000
1/1 [==============================] - 0s 0s/step - loss: 2.9907
Epoch 305/10000
1/1 [==============================] - 0s 0s/step - loss: 2.9866
Epoch 306/10000
1/1 [==============================] - 0s 0s/step - loss: 2.9825
Epoch 307/10000
1/1 [==============================] - 0s 1ms/step - loss: 2.9784
Epoch 308/10000
1/1 [==============================] - 0s 0s/step - loss: 2.9743
Epoch 309/10000
1/1 [==============================] - 0s 1ms/step - loss: 2.9702
Epoch 310/10000
1/1 [==============================] - 0s 755us/step - loss: 2.9660
Epoch 311/10000
1/1 [==============================] - 0s 1ms/step - loss: 2.9619
Epoch 312/10000
1/1 [==============================] - 0s 1ms/step - loss: 2.9578
Epoch 313/10000
1/1 [==============================] - 0s 1ms/step - loss: 2.9537
Epoch 314/10000
1/1 [===============

1/1 [==============================] - 0s 0s/step - loss: 2.5961
Epoch 403/10000
1/1 [==============================] - 0s 1ms/step - loss: 2.5923
Epoch 404/10000
1/1 [==============================] - 0s 0s/step - loss: 2.5884
Epoch 405/10000
1/1 [==============================] - 0s 1ms/step - loss: 2.5846
Epoch 406/10000
1/1 [==============================] - 0s 0s/step - loss: 2.5808
Epoch 407/10000
1/1 [==============================] - 0s 1ms/step - loss: 2.5769
Epoch 408/10000
1/1 [==============================] - 0s 1ms/step - loss: 2.5731
Epoch 409/10000
1/1 [==============================] - 0s 1ms/step - loss: 2.5693
Epoch 410/10000
1/1 [==============================] - 0s 1ms/step - loss: 2.5655
Epoch 411/10000
1/1 [==============================] - 0s 1ms/step - loss: 2.5617
Epoch 412/10000
1/1 [==============================] - 0s 1ms/step - loss: 2.5579
Epoch 413/10000
1/1 [==============================] - 0s 1ms/step - loss: 2.5541
Epoch 414/10000
1/1 [==============

...

1/1 [==============================] - 0s 1ms/step - loss: 0.6021
Epoch 4369/10000
1/1 [==============================] - 0s 1ms/step - loss: 0.6020
Epoch 4370/10000
1/1 [==============================] - 0s 1ms/step - loss: 0.6019
Epoch 4371/10000
1/1 [==============================] - 0s 1ms/step - loss: 0.6017
Epoch 4372/10000
1/1 [==============================] - 0s 0s/step - loss: 0.6016
Epoch 4373/10000
1/1 [==============================] - 0s 0s/step - loss: 0.6015
Epoch 4374/10000
1/1 [==============================] - 0s 0s/step - loss: 0.6014
Epoch 4375/10000
1/1 [==============================] - 0s 1ms/step - loss: 0.6013
Epoch 4376/10000
1/1 [==============================] - 0s 1ms/step - loss: 0.6012
Epoch 4377/10000
1/1 [==============================] - 0s 0s/step - loss: 0.6011
Epoch 4378/10000
1/1 [==============================] - 0s 0s/step - loss: 0.6009
Epoch 4379/10000
1/1 [==============================] - 0s 1ms/step - loss: 0.6008
Epoch 4380/10000
1/1 [====

1/1 [==============================] - 0s 1ms/step - loss: 0.5909
Epoch 4468/10000
1/1 [==============================] - 0s 1ms/step - loss: 0.5908
Epoch 4469/10000
1/1 [==============================] - 0s 0s/step - loss: 0.5906
Epoch 4470/10000
1/1 [==============================] - 0s 1ms/step - loss: 0.5905
Epoch 4471/10000
1/1 [==============================] - 0s 0s/step - loss: 0.5904
Epoch 4472/10000
1/1 [==============================] - 0s 0s/step - loss: 0.5903
Epoch 4473/10000
1/1 [==============================] - 0s 0s/step - loss: 0.5902
Epoch 4474/10000
1/1 [==============================] - 0s 0s/step - loss: 0.5901
Epoch 4475/10000
1/1 [==============================] - 0s 0s/step - loss: 0.5900
Epoch 4476/10000
1/1 [==============================] - 0s 0s/step - loss: 0.5899
Epoch 4477/10000
1/1 [==============================] - 0s 1ms/step - loss: 0.5898
Epoch 4478/10000
1/1 [==============================] - 0s 0s/step - loss: 0.5896
Epoch 4479/10000
1/1 [=======

1/1 [==============================] - 0s 1ms/step - loss: 0.5801
Epoch 4567/10000
1/1 [==============================] - 0s 0s/step - loss: 0.5800
Epoch 4568/10000
1/1 [==============================] - 0s 0s/step - loss: 0.5799
Epoch 4569/10000
1/1 [==============================] - 0s 0s/step - loss: 0.5798
Epoch 4570/10000
1/1 [==============================] - 0s 0s/step - loss: 0.5797
Epoch 4571/10000
1/1 [==============================] - 0s 1ms/step - loss: 0.5796
Epoch 4572/10000
1/1 [==============================] - 0s 1ms/step - loss: 0.5795
Epoch 4573/10000
1/1 [==============================] - 0s 1ms/step - loss: 0.5794
Epoch 4574/10000
1/1 [==============================] - 0s 1ms/step - loss: 0.5793
Epoch 4575/10000
1/1 [==============================] - 0s 0s/step - loss: 0.5792
Epoch 4576/10000
1/1 [==============================] - 0s 1ms/step - loss: 0.5790
Epoch 4577/10000
1/1 [==============================] - 0s 1ms/step - loss: 0.5789
Epoch 4578/10000
1/1 [====

1/1 [==============================] - 0s 0s/step - loss: 0.5698
Epoch 4666/10000
1/1 [==============================] - 0s 1ms/step - loss: 0.5697
Epoch 4667/10000
1/1 [==============================] - 0s 1ms/step - loss: 0.5696
Epoch 4668/10000
1/1 [==============================] - 0s 0s/step - loss: 0.5695
Epoch 4669/10000
1/1 [==============================] - 0s 0s/step - loss: 0.5694
Epoch 4670/10000
1/1 [==============================] - 0s 0s/step - loss: 0.5693
Epoch 4671/10000
1/1 [==============================] - 0s 1ms/step - loss: 0.5692
Epoch 4672/10000
1/1 [==============================] - 0s 0s/step - loss: 0.5691
Epoch 4673/10000
1/1 [==============================] - 0s 1ms/step - loss: 0.5690
Epoch 4674/10000
1/1 [==============================] - 0s 1ms/step - loss: 0.5689
Epoch 4675/10000
1/1 [==============================] - 0s 1ms/step - loss: 0.5688
Epoch 4676/10000
1/1 [==============================] - 0s 0s/step - loss: 0.5687
Epoch 4677/10000
1/1 [=====

1/1 [==============================] - 0s 1ms/step - loss: 0.5600
Epoch 4765/10000
1/1 [==============================] - 0s 1ms/step - loss: 0.5599
Epoch 4766/10000
1/1 [==============================] - 0s 0s/step - loss: 0.5598
Epoch 4767/10000
1/1 [==============================] - 0s 0s/step - loss: 0.5597
Epoch 4768/10000
1/1 [==============================] - 0s 0s/step - loss: 0.5596
Epoch 4769/10000
1/1 [==============================] - 0s 1ms/step - loss: 0.5595
Epoch 4770/10000
1/1 [==============================] - 0s 1ms/step - loss: 0.5594
Epoch 4771/10000
1/1 [==============================] - 0s 1ms/step - loss: 0.5593
Epoch 4772/10000
1/1 [==============================] - 0s 1ms/step - loss: 0.5592
Epoch 4773/10000
1/1 [==============================] - 0s 0s/step - loss: 0.5591
Epoch 4774/10000
1/1 [==============================] - 0s 1ms/step - loss: 0.5590
Epoch 4775/10000
1/1 [==============================] - 0s 1ms/step - loss: 0.5589
Epoch 4776/10000
1/1 [===

## Checking the Seq2Seq

In [ ]:
encoder_model = keras.Model(encoder_inputs,encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_input = [decoder_state_input_h,decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder(decoder_inputs, initial_state=decoder_states_input)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model([decoder_inputs]+decoder_states_input,
                           [decoder_outputs]+decoder_states)

In [ ]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    print(states_value)

    target_seq = np.zeros((1, 1, num_tokens))
    target_seq[0, 0, 0] = 1.

    decoded_sentence = []
    for _ in range(maxlen):
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        decoded_sentence.append(sampled_token_index)

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
decode_sequence(tokens_1hot_array[1:2,:,:])

In [ ]:
decode_sequence(tokens_1hot_array[3:4,:,:])

## Saving the Seq2Seq

Saving the model weights